# Examen Abril:

### Gramatica:
entrada-> ID "=" datos ";" | PRINT datos ";"<br>
datos-> datos OR masdatos | masdatos<br>
masdatos-> masdatos AND masmasdatos | masmasdatos<br>
masmasdatos-> NOT masmasdatos | operadores<br>
operadores-> ID | FALSE | TRUE | "(" datos ")"<br>

In [1]:
from sly import Lexer, Parser
import sys

tabla={}

In [2]:
#AST:
class Nodo():
    pass

class NodoEntradaId(Nodo):
    def hacer(a,b):
        tabla[a]=b
        
class NodoEntradaPrint(Nodo):
    def hacer(a):
        print(a)
            
class NodoOr(Nodo):
    def hacer(a,b):
        if(a==0 and b==0):
            return 0
        else:
            return 1

class NodoAnd(Nodo):
    def hacer(a,b):
        if(a==1 and b==1):
            return 1
        else:
            return 0
    
class NodoNot(Nodo):
    def hacer(a):
        if(a==1):
            return 0
        else:
            return 1

class NodoUp(Nodo):
    def hacer(a):
        return a
    
class NodoTrue(Nodo):
    def hacer(a):
        return 1
    
class NodoFalse(Nodo):
    def hacer(a):
        return 0

class NodoId(Nodo):
    def hacer(a):
        if a in tabla:
            return tabla[a]
        else:
            print('Eror.')
            print(a)
            sys.exit(0)

In [3]:
#Lexer:
class CalcLexer(Lexer):
    tokens={ID, PRINT, OR, AND, NOT, FALSE, TRUE}
    literals={'=', ';', '(', ')'}
    ignore= r'[ \t]'
    
    OR=r'or'
    AND=r'and'
    NOT=r'not'
    TRUE=r'True'
    FALSE=r'false'
    PRINT=r'print'
    
    ID=r'[a-zA-Z][a-zA-Z0-9]*'
    
    def ID(self, f):
        f.value = str(f.value)
        return f
    
    def error(self, t):
        print('Bad character %r' % t.value[0])
        self.index+=1

In [4]:
#Parser:
class CalcParser(Parser):
    tokens=CalcLexer.tokens
    def __init__(self):
        self.names={}
        
    @_('ID "=" datos ";"')
    def entrada(self, f):
        return NodoEntradaId.hacer(f.ID, f.datos)
    
    @_('PRINT datos ";"')
    def entrada(self, f):
        return NodoEntradaPrint.hacer(f.datos)
    
    @_('datos OR masdatos')
    def datos(self, f):
        return NodoOr.hacer(f.datos, f.masdatos)
    
    @_('masdatos')
    def datos(self, f):
        return NodoUp.hacer(f.masdatos)
    
    @_('masdatos AND masmasdatos')
    def masdatos(self, f):
        return NodoAnd.hacer(f.masdatos, f.masmasdatos)
    
    @_('masmasdatos')
    def masdatos(self, f):
        return NodoUp.hacer(f.masmasdatos)
    
    @_('NOT masmasdatos')
    def masmasdatos(self, f):
        return NodoNot.hacer(f.masmasdatos)
    
    @_('operadores')
    def masmasdatos(self, f):
        return NodoUp.hacer(f.operadores)
    
    @_('ID')
    def operadores(self, f):
        return NodoId.hacer(f.ID)
    
    @_('FALSE')
    def operadores(self, f):
        return NodoFalse.hacer(f.FALSE)
    
    @_('TRUE')
    def operadores(self, f):
        return NodoTrue.hacer(f.TRUE)
    
    @_('"(" datos ")"')
    def operadores(self, f):
        return NodoUp.hacer(f.datos)

In [ ]:
if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    while True:
        try:
            text = input('one expression plix > ')
        except EOFError:
            break
        if text:
            parser.parse(lexer.tokenize(text))

one expression plix > a= True;
one expression plix > print a;
1
